<img src="https://geodecode.com.br/wp-content/uploads/2021/12/brasildatacube.png" align="left" width="74"/>
<img src="https://brazil-data-cube.github.io/_images/logo-odc-bdc.png" align="right" width="74" />

<h1 style="color:#336699; text-align: center" >Annual Metting Use Case</h1>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    <sup>
        <a href="https://orcid.org/0000-0002-9916-1167">
            <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i>
        </a>
    </sup>
    <br/>
    Brazil Data Cube (BDC), National Institute for Space Research (INPE)
    <br/>
    <br/>
    <br/>
    <div style="width: 98%;">
        <div style="text-align: justify; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 10px;">
            <b>Explantion here....:</b>
            <div>
                Write write write write writewrite writewrite write. <a href="http://www.brazildatacube.org/" target="_blank">Site site site site site</a>, <i> Outras infos</i> .......
            </div>
        </div>
    </div>
</div>

In [1]:
library(sf)
library(rstac)
library(dplyr)

Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 8.2.1; sf_use_s2() is TRUE


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Set constants
<hr style="border:1px solid #0077b9;">

In [15]:
HARMONIZE_STAC_URL <- 'https://brazildatacube.dpi.inpe.br/harmonize/dev/stac/v1/'

FINAL_CSV_PREFIX <- "data_2020_week_"

START_DATE <- "2020-01-01"
# END_DATE <- "2021-12-31"
END_DATE <- "2020-12-31"

# List Harmonize STAC collections (using RSTAC)
<hr style="border:1px solid #0077b9;">

In [16]:
all_collections <- stac(HARMONIZE_STAC_URL) %>%
  collections() %>%
  get_request()

for (collection in all_collections$collections){
  print(collection$id)
}

[1] "dengue_mortality_rate_mun_year_north-1"
[1] "max_temper_mean_ne_mun_month-1"
[1] "total_prec_max_no_mun_epiweek-1"
[1] "Phantom3Advanced_FlightHeight120m_Mosaic-1"
[1] "Phantom3Advanced_FlightHeight120m-1"
[1] "chagas_disease_cases_mun_week_north-1"
[1] "max_temper_mean_no_mun_month-1"
[1] "max_temper_max_no_mun_month-1"
[1] "max_temper_max_no_mun_epiweek_ras-1"
[1] "chagas_disease_cases_mun_month_north-1"
[1] "max_temper_max_ne_mun_epiweek-1"
[1] "dengue_cases_mun_year_north-1"
[1] "chikungunya_cases_mun_year_north-1"
[1] "zika_cases_mun_month_north-1"
[1] "dengue_cases_mun_month_north-1"
[1] "chikungunya_cases_mun_month_north-1"
[1] "dengue_cases_mun_week_north-1"
[1] "dengue_mortality_rate_mun_month_north-1"
[1] "max_temper_mean_no_mun_epiweek-1"
[1] "zika_cases_mun_year_north-1"
[1] "chagas_disease_cases_mun_year_north-1"
[1] "zika_cases_mun_week_north-1"
[1] "total_prec_max_no_mun_month-1"
[1] "dengue_mortality_rate_mun_week_north-1"
[1] "chikungunya_cases_mun_week_north-1"
[

# Obtain dengue data
<hr style="border:1px solid #0077b9;">

### Set collection id

In [17]:
# DENGUE_COLLECTION_ID <- "dengue_cases_mun_month_north-1"
# DENGUE_COLLECTION_ID <- "dengue_mortality_rate_mun_month_north-1"
DENGUE_COLLECTION_ID <- "dengue_cases_mun_week_north-1"

DATA_TYPE <- "dengue"
# DATA_TYPE <- "zika"

### Search Dengue data on Harmonize STAC (Using RSTAC)

In [18]:
health_search <- stac(HARMONIZE_STAC_URL) %>%
  stac_search(
    collections = DENGUE_COLLECTION_ID,
    datetime = paste(START_DATE, END_DATE, sep = "/"),
    limit = 200
  ) %>%
  get_request()

health_search$context

$matched
[1] 53

$returned
[1] 53

### For each feature, retrieve geojson as dataframe

In [19]:
health_feature_to_geojson_csv <- function(feature) {
  datetime <- feature$properties$datetime
  geojson_href <- feature$assets$geojson$href

  as.data.frame(st_read(geojson_href)) %>%
    mutate(date = format(as.Date(datetime), "%Y-%m-%d"))
}

health_dfs <- lapply(X = health_search$features, FUN = health_feature_to_geojson_csv)
health_dfs[1:2]

Reading layer `dengue_cases_mun_week_north_20201230' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/health_indicators/indi_0015/week/municipality/items/2020-12-30/north/dengue_cases_mun_week_north_20201230.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `dengue_cases_mun_week_north_20201223' from data source 
  `https://brazildatacube.dpi.inpe.br/harmonize/dev/data/health_indicators/indi_0015/week/municipality/items/2020-12-23/north/dengue_cases_mun_week_north_20201223.geojson' 
  using driver `GeoJSON'
Simple feature collection with 21 features and 6 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -50.9967 ymin: -5.196937 xmax: -48.13749 ymax: -1.019427
Geodetic CRS:  WGS 84
Reading layer `dengue_cases_mun_week_north_20201216' from data sourc

cod,date,name,agg,agg_time,value,geometry
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<POLYGON [°]>
150010,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-48.91969 -1.9472...
150080,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-48.39812 -1.3927...
150110,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-49.94238 -2.7575...
150120,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-49.55923 -2.9383...
150130,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-48.66029 -1.6923...
150140,2020-12-30,indi_0015,mun_res,week,0.4,POLYGON ((-48.52311 -1.3820...
150150,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-48.28103 -1.3827...
150178,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-49.23882 -4.0763...
150210,2020-12-30,indi_0015,mun_res,week,0.0,POLYGON ((-49.26145 -2.0259...


### Rbind lists of monthly dataframes in one big dataframe

In [20]:
health_df <- do.call(rbind, health_dfs) %>%
  dplyr::select(cod, date, value) %>%
  rename(cd_mun = cod, !!DATA_TYPE := value)

dim(health_df)
head(health_df)

[1] 1113    3

,cd_mun,date,dengue
,<chr>,<chr>,<chr>
1,150010,2020-12-30,0.0
2,150080,2020-12-30,0.0
3,150110,2020-12-30,0.0
4,150120,2020-12-30,0.0
5,150130,2020-12-30,0.0
6,150140,2020-12-30,0.4


### Save health data

In [22]:
FILENAME <- paste0(FINAL_CSV_PREFIX, DATA_TYPE, ".csv")

write.csv(health_df, FILENAME, row.names = FALSE)